In [1]:
from __future__ import division

In [2]:
import sys
sys.path.append('..')

In [3]:
import math
import random
import numpy as np
import pandas as pd

In [4]:
import gtree

In [5]:
%load_ext autoreload
%autoreload 1

%aimport gtree
%aimport tools
%aimport tree._my_tree

In [6]:
tree._my_tree.foo(10)

100.0

# The goal of this notebook is to explore ways to optimizing our splitting algorithm, which is a huge part of the cost of this code

In [15]:
from timeit import default_timer as timer

class benchmark(object):

    def __init__(self, msg, fmt="%0.3g"):
        self.msg = msg
        self.fmt = fmt

    def __enter__(self):
        self.start = timer()
        return self

    def __exit__(self, *args):
        t = timer() - self.start
        print(("%s : " + self.fmt + " seconds") % (self.msg, t))
        self.time = t

In [16]:
with benchmark("Just A test"):
    print "Foo"

Foo
Just A test : 0.000412 seconds


# Investigate Optimal Splitting

Can we do splitting faster using raw numpy arrays rather than Pandas DataFrames?

In [7]:
df_features, df_targets = tools.make_random_classification(5000)
df_features = pd.DataFrame(df_features, dtype='float32')
df_targets = pd.Series(df_targets, dtype='float32')

In [8]:
np_feaures = np.array(df_features.values, dtype='float32')
np_targets = np.array(df_targets.values, dtype='float32')

In [11]:
feature_names = df_features.columns

In [12]:
# Get the candidates in advance
candidate_map = {v: gtree._get_split_candidates(df_features[v]) for v in feature_names}

In [ ]:
loss_fn = gtree.cross_entropy_loss
prediction_builder = gtree.leaf_good_rate_prediction_builder

In [ ]:
# Create the numpy version

In [ ]:
x = pd.DataFrame({
    'a': [1, 2, 3, 4, 3, 6],
    'b': [10, 20, 30, 40, 50, 60]
})
y = pd.Series([1, 0, 0, 1, 0, 1])

In [13]:
var = 'feature_15'
var_idx = 15
candidates = set(candidate_map[var])

In [ ]:
with benchmark("GTREE DF Single"):
    s, l = gtree._df_single_variable_best_split(
            df_features, var, df_targets,
            loss_fn=loss_fn, #gtree.error_rate_loss,
            leaf_prediction_builder=prediction_builder,
            candidates = candidates)
    print s, l

In [ ]:
with benchmark("GTREE HYBRID Single"):
    s, l = gtree._hybrid_single_variable_best_split(
            df_features, var, df_targets,
            loss_fn=loss_fn, #gtree.error_rate_loss,
            leaf_prediction_builder=prediction_builder,
            candidates = candidates)
    print s, l

In [ ]:
with benchmark("GTREE NP Single"):
    s, l = gtree._np_single_variable_best_split(
            df_features, var, df_targets,
            loss_fn=loss_fn, #gtree.error_rate_loss,
            leaf_prediction_builder=prediction_builder,
            candidates = candidates)
    print s, l

In [20]:
lmb = tree._my_tree.MeanLeafMapperBuilder()
ce = tree._my_tree.CrossEntropyLoss()
#spliter = tree._my_tree.SpitFinder()

with benchmark("NP CYTHON"):
    s, l = tree._my_tree.getBestSplit(
        np_feaures,
        var_idx,
        np_targets,
        gtree.cross_entropy_loss,
        gtree.leaf_good_rate_prediction_builder,
        candidates)
    print s, l

-0.807416439056 0.692991316319
NP CYTHON : 0.0296 seconds


In [21]:
#lmb = tree._my_tree.MeanLeafMapperBuilder()
#ce = tree._my_tree.ErrorRateLoss()
#spliter = tree._my_tree.SpitFinder()

with benchmark("NP CYTHON ERROR RATE LOSS"):
    s, l = tree._my_tree.getBestSplit(
        np_feaures,
        var_idx,
        np_targets,
        gtree.error_rate_loss,
        gtree.leaf_good_rate_prediction_builder,
        candidates)
    print s, l

0.251264750957 0.50159996748
NP CYTHON ERROR RATE LOSS : 0.0202 seconds


In [ ]:
lmb = tree._my_tree.MeanLeafMapperBuilder()
ce = tree._my_tree.CrossEntropyLoss()
#spliter = tree._my_tree.SpitFinder()

with benchmark("SKLEARN ADAPTED"):
    s, l = spliter.getBestSplit(
        var_idx,
        candidates,
        np_feaures,
        np_targets,
        lmb,
        ce)
    print s, la\

In [ ]:
#with benchmark("NP NAIVE"):
#    s, l = np_naive_single_variable_best_split(
#        features=np_feaures, var_idx=var_idx, target=np_targets,
#        loss_fn=loss_fn, #gtree.error_rate_loss,
#        leaf_prediction_builder=np_leaf_good_rate_split_builder,
#        candidates=candidates)
#    print s, l

In [ ]:
#with benchmark("NP SMART"):
#    s, l = np_single_variable_best_split(
#        features=np_feaures, var_idx=15, target=np_targets,
#        loss_fn=loss_fn, #gtree.error_rate_loss,
#        leaf_prediction_builder=np_leaf_good_rate_split_builder,
#        candidates=candidates)
#    print s, l

# Test multiple splits

-------------------

In [ ]:
# Test Df Splitting
with benchmark("DF Test"):
    for var in feature_names:
        gtree._df_single_variable_best_split(
            df_features, var, df_targets,
            loss_fn=loss_fn,
            leaf_prediction_builder=prediction_builder,
            candidates = candidate_map[var])

In [ ]:
with benchmark("NP Test"):
    for var in feature_names:
        gtree._np_single_variable_best_split(
            df_features, var, df_targets,
            loss_fn=loss_fn,
            leaf_prediction_builder=prediction_builder,
            candidates = candidate_map[var])

# Misc

In [ ]:
X = np.array([[1, 7, 3],
              [4, 5, 6],
              [7, 8, 9]])
Y = np.array([1, 2, 3])

In [ ]:
sort_by_col(X, Y, 1)

In [ ]:
Y

In [ ]:
x = pd.DataFrame({
    'a': [1, 2, 3, 4, 3, 6],
    'b': [10, 20, 30, 40, 50, 60]
})
y = pd.Series([1, 0, 0, 1, 0, 1])

In [ ]:
from sklearn.tree import _criterion, _splitter

c = _criterion.Gini(1, np.array([2]))
s = _splitter.BestSplitter

In [ ]:
max_features = len(df_features)

splitter = s(c,
             max_features,
             min_samples_leaf=1,
             min_weight_leaf=1,
             random_state=1,
             presort=False)

In [ ]:
splitter.

In [ ]:
s.node_split

In [ ]:
x = pd.Series([1, 2, 3])

In [ ]:
x.index[[2, 0, 1]]